In [38]:
!pwd

/Users/mkeller/research/dbmi/vitessce/paper-figures/visium/src


In [48]:
from os.path import join
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
    OmeZarrWrapper,
    hconcat,
    vconcat,
)

In [49]:
processed_dir = join("..", "..", "..", "vitessce-python", "demos", "human-lymph-node-10x-visium", "data", "processed")

In [50]:
vc = VitessceConfig(schema_version="1.0.15", name='Visium data', description='')

In [51]:
img_zarr = join(processed_dir, "human_lymph_node_10x_visium.ome.zarr")
adata_zarr = join(processed_dir, "human_lymph_node_10x_visium.h5ad.zarr")

In [52]:
dataset = vc.add_dataset(name='Human lymph node').add_object(AnnDataWrapper(
    adata_path=adata_zarr,
    obs_locations_path="obsm/spatial",
    obs_segmentations_path="obsm/segmentations",
    obs_embedding_paths=["obsm/X_umap", "obsm/X_pca"],
    obs_embedding_names=["UMAP", "PCA"],
    obs_set_paths=["obs/clusters"],
    obs_set_names=["Leiden Cluster"],
    obs_feature_matrix_path="X",
    initial_feature_filter_path="var/highly_variable",
    # To be explicit that the features represent genes and gene expression, we specify that here.
    coordination_values={
        "obsType": "spot"
    }
)).add_object(OmeZarrWrapper(
    # We next run add_object with adata_path=adt_zarr to add the cell-by-ADT matrix and associated metadata.
    img_path=img_zarr,
))

In [53]:
spatial_by_cellset = vc.add_view(vt.SPATIAL, dataset=dataset)
spatial_by_expression = vc.add_view(vt.SPATIAL, dataset=dataset)
heatmap = vc.add_view(vt.HEATMAP, dataset=dataset).set_props(transpose=True)
lc = vc.add_view(vt.LAYER_CONTROLLER, dataset=dataset).set_props(disableChannelsIfRgbDetected=True)
cell_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)
feature_list = vc.add_view(vt.FEATURE_LIST, dataset=dataset)

all_views = [
    spatial_by_cellset,
    spatial_by_expression,
    heatmap,
    lc,
    cell_sets,
    feature_list,
]

segmentation_layer = {
    "radius": 65, "stroked": True, "visible": True, "opacity": 1
}
image_layer = [
    {
      "type": "raster",
      "index": 0,
      "colormap": None,
      "transparentColor": None,
      "opacity": 1,
      "domainType": "Min/Max",
      "channels": [
        {
          "selection": { "c": 0 },
          "color": [
            255,
            0,
            0
          ],
          "visible": True,
          "slider": [
            0,
            255
          ]
        },
        {
          "selection": { "c": 1 },
          "color": [
            0,
            255,
            0
          ],
          "visible": True,
          "slider": [
            0,
            255
          ]
        },
        {
          "selection": { "c": 2 },
          "color": [
            0,
            0,
            255
          ],
          "visible": True,
          "slider": [
            0,
            255
          ]
        }
      ]
    }
]

vc.link_views(all_views, [ct.OBS_TYPE], ["spot"])
vc.link_views([spatial_by_cellset, spatial_by_expression, lc], [ct.SPATIAL_SEGMENTATION_LAYER, ct.SPATIAL_IMAGE_LAYER, ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y], [segmentation_layer, image_layer, -2.598, 1008.88, 1004.69])
vc.link_views([spatial_by_expression, feature_list], [ct.OBS_COLOR_ENCODING, ct.FEATURE_SELECTION], ["geneSelection", ["CR2"]])
vc.link_views([spatial_by_cellset, cell_sets], [ct.OBS_COLOR_ENCODING], ["cellSetSelection"])


vc.layout((spatial_by_cellset | spatial_by_expression) / (hconcat(lc, cell_sets, feature_list) | heatmap));

In [54]:
vc.web_app()

'http://vitessce.io/#?theme=light&url=data:,%7B%22version%22%3A+%221.0.15%22%2C+%22name%22%3A+%22Visium+data%22%2C+%22description%22%3A+%22%22%2C+%22datasets%22%3A+%5B%7B%22uid%22%3A+%22A%22%2C+%22name%22%3A+%22Human+lymph+node%22%2C+%22files%22%3A+%5B%7B%22fileType%22%3A+%22anndata.zarr%22%2C+%22url%22%3A+%22http%3A%2F%2Flocalhost%3A8009%2FA%2F0%2F866d513d-2dc7-4e16-b268-6bafdbe1aa62%22%2C+%22options%22%3A+%7B%22obsLocations%22%3A+%7B%22path%22%3A+%22obsm%2Fspatial%22%7D%2C+%22obsSegmentations%22%3A+%7B%22path%22%3A+%22obsm%2Fsegmentations%22%7D%2C+%22obsEmbedding%22%3A+%5B%7B%22path%22%3A+%22obsm%2FX_umap%22%2C+%22dims%22%3A+%5B0%2C+1%5D%2C+%22embeddingType%22%3A+%22UMAP%22%7D%2C+%7B%22path%22%3A+%22obsm%2FX_pca%22%2C+%22dims%22%3A+%5B0%2C+1%5D%2C+%22embeddingType%22%3A+%22PCA%22%7D%5D%2C+%22obsSets%22%3A+%5B%7B%22name%22%3A+%22Leiden+Cluster%22%2C+%22path%22%3A+%22obs%2Fclusters%22%7D%5D%2C+%22obsFeatureMatrix%22%3A+%7B%22path%22%3A+%22X%22%2C+%22initialFeatureFilterPath%22%3A+%22va